# Cleaning KS Table

In [85]:
import pandas as pd
import sql_functions as sf
import capstone_json_functions as cjf

engine = sf.get_engine()
 
schema = "bgg_data"

In [86]:
sql = f'''SELECT * FROM {schema}.unique_slug_bgg_id;
      '''
df_bgg_ks = sf.get_dataframe(sql)

In [87]:
sql = f'''SELECT * FROM {schema}.kickstarter_raw;
      '''
df = sf.get_dataframe(sql)

In [88]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41975 entries, 0 to 41974
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   game_name         41975 non-null  object        
 1   goal              41975 non-null  int64         
 2   pledged           41975 non-null  float64       
 3   state             41975 non-null  object        
 4   slug              41975 non-null  object        
 5   country           41975 non-null  object        
 6   currency          41975 non-null  object        
 7   deadline          41975 non-null  datetime64[ns]
 8   created_at        41975 non-null  datetime64[ns]
 9   launched_at       41975 non-null  datetime64[ns]
 10  backers_count     41975 non-null  int64         
 11  usd_pledged       41975 non-null  float64       
 12  creator_name      41975 non-null  object        
 13  state_changed_at  41975 non-null  datetime64[ns]
dtypes: datetime64[ns](4), 

## Transform state table in a boolean value

In [89]:
df['successful'] = df['state'] == 'successful'

In [90]:
df.drop('state',axis=1,inplace=True)

### Transform date in unix timestamp

In [91]:
df = cjf.datetime_to_unix(df)

## Remove multiple values for same campaign

We have used multiple historical data from different scraping day and we want to key only the last value for every kickstarter campaign.

In [92]:
df_filtered = df.groupby(['slug','deadline','created_at','launched_at','country','currency']).max('state_changed_at')

In [93]:
df_filtered

,,,,,,goal,pledged,backers_count,usd_pledged,state_changed_at,successful
slug,deadline,created_at,launched_at,country,currency,,,,,,
0-hr-starship-maps-and-miniatures,1.521947e+09,1.520618e+09,1.520885e+09,US,USD,1800,9934.00,108,9934.00,1.521947e+09,True
0-hr-tri-toad-poster-and-starship-sale,1.612325e+09,1.606859e+09,1.611157e+09,US,USD,2000,10625.00,227,10625.00,1.612325e+09,True
1-48tactic-cards-only,1.584194e+09,1.582716e+09,1.583593e+09,IT,EUR,950,7861.00,104,8828.77,1.584194e+09,True
1-48tactic-cards-only-ww2-skirmish-game-gets-new-factions,1.605804e+09,1.603295e+09,1.604076e+09,IT,EUR,2500,12330.32,78,14491.09,1.605804e+09,True
1-48tactic-cards-only-ww2-skirmish-gaming-goes-global,1.603978e+09,1.590246e+09,1.601386e+09,IT,EUR,9500,6266.00,38,7290.93,1.601386e+09,False
...,...,...,...,...,...,...,...,...,...,...,...
zweihander-grim-and-perilous-rpg,1.472137e+09,1.468519e+09,1.469545e+09,US,USD,7100,61743.00,1265,61743.00,1.472137e+09,True
zweihander-rpg-starter-kit,1.646068e+09,1.639413e+09,1.644858e+09,US,USD,10000,82119.00,1460,82119.00,1.646068e+09,True
zynvaded-evo,1.472904e+09,1.467144e+09,1.470312e+09,US,USD,7700,8499.00,81,8499.00,1.472904e+09,True


In [120]:
df_filtered = cjf.unix_to_datetime(df_filtered,['deadline','created_at','launched_at','state_changed_at'])

In [121]:
df_filtered

,slug,deadline,created_at,launched_at,country,currency,goal,pledged,backers_count,usd_pledged,state_changed_at,successful
0,0-hr-starship-maps-and-miniatures,2018-03-25 03:00:00,2018-03-09 17:51:22,2018-03-12 19:58:06,US,USD,1800,9934.00,108,9934.00,2018-03-25 03:00:00,True
1,0-hr-tri-toad-poster-and-starship-sale,2021-02-03 04:00:00,2020-12-01 21:35:30,2021-01-20 15:38:46,US,USD,2000,10625.00,227,10625.00,2021-02-03 04:00:00,True
2,1-48tactic-cards-only,2020-03-14 14:00:39,2020-02-26 11:26:46,2020-03-07 15:00:39,IT,EUR,950,7861.00,104,8828.77,2020-03-14 14:00:39,True
3,1-48tactic-cards-only-ww2-skirmish-game-gets-n...,2020-11-19 16:43:03,2020-10-21 15:43:12,2020-10-30 16:43:03,IT,EUR,2500,12330.32,78,14491.09,2020-11-19 16:43:03,True
4,1-48tactic-cards-only-ww2-skirmish-gaming-goes...,2020-10-29 13:30:01,2020-05-23 15:03:13,2020-09-29 13:30:01,IT,EUR,9500,6266.00,38,7290.93,2020-09-29 13:30:05,False
...,...,...,...,...,...,...,...,...,...,...,...,...
26176,zweihander-grim-and-perilous-rpg,2016-08-25 14:59:22,2016-07-14 18:00:41,2016-07-26 14:59:22,US,USD,7100,61743.00,1265,61743.00,2016-08-25 14:59:22,True
26177,zweihander-rpg-starter-kit,2022-02-28 17:00:06,2021-12-13 16:32:16,2022-02-14 17:00:06,US,USD,10000,82119.00,1460,82119.00,2022-02-28 17:00:09,True
26178,zynvaded-evo,2016-09-03 12:01:36,2016-06-28 19:57:34,2016-08-04 12:01:36,US,USD,7700,8499.00,81,8499.00,2016-09-03 12:01:37,True
26179,zynvaded-evo-presents-dont-let-the-zed-bugz-bite,2021-12-26 15:35:29,2021-10-05 01:18:42,2021-11-26 15:35:29,US,USD,1000,5156.00,39,5156.00,2021-12-26 15:35:29,True


In [122]:
# sf.build_table(engine=engine, table_name='kickstarter_unique_campaigns',dataframe=df_filtered,schema=schema)

The kickstarter_unique_campaigns table was imported successfully.


In [95]:
df_filtered = df_filtered.reset_index()

In [96]:
mask = df_filtered['slug'] == 'f-life-a-card-game-about-adulthood'

In [97]:
df_filtered.loc[mask,:]

,slug,deadline,created_at,launched_at,country,currency,goal,pledged,backers_count,usd_pledged,state_changed_at,successful
8278,f-life-a-card-game-about-adulthood,1.476972e+09,1.447869e+09,1.473948e+09,US,USD,20000,528.0,18,528.0,1.473948e+09,False


In [98]:
df_filtered

,slug,deadline,created_at,launched_at,country,currency,goal,pledged,backers_count,usd_pledged,state_changed_at,successful
0,0-hr-starship-maps-and-miniatures,1.521947e+09,1.520618e+09,1.520885e+09,US,USD,1800,9934.00,108,9934.00,1.521947e+09,True
1,0-hr-tri-toad-poster-and-starship-sale,1.612325e+09,1.606859e+09,1.611157e+09,US,USD,2000,10625.00,227,10625.00,1.612325e+09,True
2,1-48tactic-cards-only,1.584194e+09,1.582716e+09,1.583593e+09,IT,EUR,950,7861.00,104,8828.77,1.584194e+09,True
3,1-48tactic-cards-only-ww2-skirmish-game-gets-n...,1.605804e+09,1.603295e+09,1.604076e+09,IT,EUR,2500,12330.32,78,14491.09,1.605804e+09,True
4,1-48tactic-cards-only-ww2-skirmish-gaming-goes...,1.603978e+09,1.590246e+09,1.601386e+09,IT,EUR,9500,6266.00,38,7290.93,1.601386e+09,False
...,...,...,...,...,...,...,...,...,...,...,...,...
26176,zweihander-grim-and-perilous-rpg,1.472137e+09,1.468519e+09,1.469545e+09,US,USD,7100,61743.00,1265,61743.00,1.472137e+09,True
26177,zweihander-rpg-starter-kit,1.646068e+09,1.639413e+09,1.644858e+09,US,USD,10000,82119.00,1460,82119.00,1.646068e+09,True
26178,zynvaded-evo,1.472904e+09,1.467144e+09,1.470312e+09,US,USD,7700,8499.00,81,8499.00,1.472904e+09,True
26179,zynvaded-evo-presents-dont-let-the-zed-bugz-bite,1.640533e+09,1.633397e+09,1.637941e+09,US,USD,1000,5156.00,39,5156.00,1.640533e+09,True


In [99]:
df_filtered.describe()

,deadline,created_at,launched_at,goal,pledged,backers_count,usd_pledged,state_changed_at
count,2.618100e+04,2.618100e+04,2.618100e+04,2.618100e+04,2.618100e+04,26181.000000,2.618100e+04,2.618100e+04
mean,1.534218e+09,1.524838e+09,1.531801e+09,1.602731e+04,6.709881e+04,664.798021,5.183862e+04,1.533564e+09
std,7.986241e+07,8.086975e+07,8.002183e+07,9.428855e+04,4.783484e+05,2564.758369,2.753624e+05,7.976403e+07
min,1.245043e+09,1.242364e+09,1.242370e+09,1.000000e+00,0.000000e+00,0.000000,0.000000e+00,1.245043e+09
25%,1.479067e+09,1.469560e+09,1.476549e+09,1.000000e+03,1.840000e+03,49.000000,1.825980e+03,1.478012e+09
50%,1.544224e+09,1.535749e+09,1.542028e+09,5.000000e+03,6.829290e+03,156.000000,6.681140e+03,1.543572e+09
75%,1.599834e+09,1.590646e+09,1.597464e+09,1.250000e+04,2.473260e+04,479.000000,2.345443e+04,1.599222e+09
max,1.659885e+09,1.654693e+09,1.654733e+09,8.547100e+06,3.202978e+07,219382.000000,1.296961e+07,1.654740e+09


In [100]:
df_filtered.sort_values('usd_pledged',ascending=False)

,slug,deadline,created_at,launched_at,country,currency,goal,pledged,backers_count,usd_pledged,state_changed_at,successful
9341,frosthaven,1.588367e+09,1.581300e+09,1.585667e+09,US,USD,500000,12969608.00,83193,12969608.00,1.588367e+09,True
12343,kingdom-death-monster-15,1.483830e+09,1.478708e+09,1.480050e+09,US,USD,100000,12393139.69,19264,12393139.69,1.483830e+09,True
13741,marvel-zombies-zombicide,1.643940e+09,1.629686e+09,1.642536e+09,US,USD,500000,9032583.00,28974,9032583.00,1.643940e+09,True
14474,modular-gaming-table,1.602302e+09,1.592863e+09,1.597168e+09,US,USD,1000000,8808136.00,7983,8808136.00,1.602302e+09,True
8233,exploding-kittens,1.424398e+09,1.418633e+09,1.421777e+09,US,USD,10000,8782571.99,219382,8782571.99,1.424398e+09,True
...,...,...,...,...,...,...,...,...,...,...,...,...
25616,woodlands-sigil-of-love-oracle-card-board-game...,1.524400e+09,1.521478e+09,1.521808e+09,US,USD,50000,0.00,0,0.00,1.521808e+09,False
2119,awesome-rapping-skeletal-planchette-for-ouija-...,1.447477e+09,1.444005e+09,1.444075e+09,US,USD,666,0.00,0,0.00,1.444075e+09,False
2833,bilingual-dice,1.447451e+09,1.438463e+09,1.442264e+09,US,USD,160000,0.00,0,0.00,1.442264e+09,False
20805,tactical-warchest-card-game-position-your-unit...,1.522510e+09,1.515386e+09,1.519922e+09,MX,MXN,700000,0.00,0,0.00,1.519922e+09,False


# Merge KS table with BGG ids

In [101]:
df_merged =  pd.merge(df_bgg_ks,df_filtered,on='slug')

In [102]:
df_merged.nunique()

slug                4520
bgg_id              4405
deadline            4489
created_at          4520
launched_at         4518
country               24
currency              14
goal                 449
pledged             4368
backers_count       1858
usd_pledged         4439
state_changed_at    4494
successful             2
dtype: int64

In [103]:
mask = df_merged['bgg_id'] == 331754
df_merged.loc[mask,:]

,slug,bgg_id,deadline,created_at,launched_at,country,currency,goal,pledged,backers_count,usd_pledged,state_changed_at,successful
2689,f-life-a-card-game-about-adulthood-1,331754,1.499954e+09,1.496901e+09,1.496930e+09,US,USD,10000,10621.0,131,10621.0,1.499954e+09,True
2690,f-life-a-card-game-about-adulthood,331754,1.476972e+09,1.447869e+09,1.473948e+09,US,USD,20000,528.0,18,528.0,1.473948e+09,False
2691,f-life-a-card-game-about-adulthood-0,331754,1.496873e+09,1.480985e+09,1.493849e+09,US,USD,16000,1920.0,60,1920.0,1.493849e+09,False


In [104]:
df_merged.head(50)

,slug,bgg_id,deadline,created_at,launched_at,country,currency,goal,pledged,backers_count,usd_pledged,state_changed_at,successful
0,2kings,319727,1.606403e+09,1.600172e+09,1.603807e+09,IT,EUR,15000,3431.00,112,4060.16,1.603807e+09,False
1,9-lives,267618,1.551618e+09,1.548825e+09,1.549026e+09,SG,SGD,13512,18021.00,175,13385.64,1.551618e+09,True
2,adorable-monsters-0,231388,1.505229e+09,1.504725e+09,1.504883e+09,US,USD,250,2985.00,92,2985.00,1.505229e+09,True
3,aftermath-5,231388,1.579582e+09,1.574394e+09,1.574398e+09,US,USD,10000,229.00,8,229.00,1.574398e+09,False
4,alibi,209538,1.446319e+09,1.441232e+09,1.443727e+09,US,USD,5400,5710.69,84,5710.69,1.446319e+09,True
5,aurora-2,134567,1.637330e+09,1.622581e+09,1.634735e+09,US,USD,4000,5619.00,110,5619.00,1.637330e+09,True
6,baby-blues,348554,1.464811e+09,1.463620e+09,1.464217e+09,US,USD,1000,2871.00,195,2871.00,1.464811e+09,True
7,battle-of-thermopylae,204003,1.646154e+09,1.643272e+09,1.644340e+09,ES,EUR,900,8834.00,419,10105.01,1.646155e+09,True
8,bingo-dice,217430,1.542600e+09,1.531404e+09,1.540313e+09,DE,EUR,2400,579.00,19,666.70,1.540313e+09,False
9,black-swan-0,301946,1.594127e+09,1.590588e+09,1.592831e+09,IT,EUR,30000,62742.00,447,70128.96,1.594127e+09,True


## I want to count how many KS campaigns have a BGG id

In [105]:
df_merged_grouped = df_merged.groupby('bgg_id').count()['slug'].sort_values(ascending=False)

In [106]:
df_merged_grouped

bgg_id
331754    3
246267    2
225740    2
224937    2
224594    2
         ..
208512    1
208543    1
208568    1
208569    1
365503    1
Name: slug, Length: 4405, dtype: int64

## I create a pandas Series of BBG games that had more than one Kickstarter campaign

In [107]:
df_merged_grouped = df_merged_grouped.reset_index()

In [108]:
mask = df_merged_grouped['slug'] > 1 

bgg_ids_with_more_slugs = df_merged_grouped.loc[mask,'bgg_id']

In [109]:
bgg_ids_with_more_slugs

0      331754
1      246267
2      225740
3      224937
4      224594
        ...  
126    249689
127     11670
128    249748
129    364399
130     19464
Name: bgg_id, Length: 131, dtype: int64

## I want also to aggregate the KS slugs that refer to the same BGG ID and keep only the most recent

In [110]:
df_merged.head(2)

,slug,bgg_id,deadline,created_at,launched_at,country,currency,goal,pledged,backers_count,usd_pledged,state_changed_at,successful
0,2kings,319727,1.606403e+09,1.600172e+09,1.603807e+09,IT,EUR,15000,3431.0,112,4060.16,1.603807e+09,False
1,9-lives,267618,1.551618e+09,1.548825e+09,1.549026e+09,SG,SGD,13512,18021.0,175,13385.64,1.551618e+09,True


In [111]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4537 entries, 0 to 4536
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   slug              4537 non-null   object 
 1   bgg_id            4537 non-null   int64  
 2   deadline          4537 non-null   float64
 3   created_at        4537 non-null   float64
 4   launched_at       4537 non-null   float64
 5   country           4537 non-null   object 
 6   currency          4537 non-null   object 
 7   goal              4537 non-null   int64  
 8   pledged           4537 non-null   float64
 9   backers_count     4537 non-null   int64  
 10  usd_pledged       4537 non-null   float64
 11  state_changed_at  4537 non-null   float64
 12  successful        4537 non-null   bool   
dtypes: bool(1), float64(6), int64(3), object(3)
memory usage: 465.2+ KB


I need first to convert the date in unix to get max value

In [112]:
df_clean_bgg_id_ks_slug = cjf.datetime_to_unix(df_merged)

In [113]:
df_clean_bgg_id_ks_slug

,slug,bgg_id,deadline,created_at,launched_at,country,currency,goal,pledged,backers_count,usd_pledged,state_changed_at,successful
0,2kings,319727,1.606403e+09,1.600172e+09,1.603807e+09,IT,EUR,15000,3431.00,112,4060.16,1.603807e+09,False
1,9-lives,267618,1.551618e+09,1.548825e+09,1.549026e+09,SG,SGD,13512,18021.00,175,13385.64,1.551618e+09,True
2,adorable-monsters-0,231388,1.505229e+09,1.504725e+09,1.504883e+09,US,USD,250,2985.00,92,2985.00,1.505229e+09,True
3,aftermath-5,231388,1.579582e+09,1.574394e+09,1.574398e+09,US,USD,10000,229.00,8,229.00,1.574398e+09,False
4,alibi,209538,1.446319e+09,1.441232e+09,1.443727e+09,US,USD,5400,5710.69,84,5710.69,1.446319e+09,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4532,dark-venture-battle-of-the-ancients,298163,1.606252e+09,1.578407e+09,1.603808e+09,US,USD,14000,53696.00,953,53696.00,1.606252e+09,True
4533,rhetorical-oracle-0,298189,1.581808e+09,1.578162e+09,1.579087e+09,US,USD,6000,311.00,8,311.00,1.579087e+09,False
4534,beezarre-games-fox-on-the-run,298193,1.594915e+09,1.562243e+09,1.592323e+09,IT,EUR,9600,12101.00,298,13630.08,1.594915e+09,True
4535,bellum-0,217430,1.488402e+09,1.485964e+09,1.486480e+09,FR,EUR,4000,8037.00,307,8660.22,1.488402e+09,True


In [114]:
df_clean_bgg_id_ks_slug = df_merged.groupby(['bgg_id','country','currency']).max(['launched_at','deadline','created_at'])



In [115]:
df_clean_bgg_id_ks_slug = df_clean_bgg_id_ks_slug.reset_index()

In [116]:
df_clean_bgg_id_ks_slug

,bgg_id,country,currency,deadline,created_at,launched_at,goal,pledged,backers_count,usd_pledged,state_changed_at,successful
0,37,NL,EUR,1.551474e+09,1.548605e+09,1.550137e+09,4750,14132.00,161,16021.28,1.551474e+09,True
1,46,US,USD,1.511755e+09,1.504062e+09,1.508437e+09,18446,76254.00,980,76254.00,1.511755e+09,True
2,196,US,USD,1.373463e+09,1.366126e+09,1.370871e+09,9000,53724.58,1447,53724.58,1.373463e+09,True
3,257,US,USD,1.447474e+09,1.444406e+09,1.444763e+09,40000,210086.50,2711,210086.50,1.447474e+09,True
4,470,US,USD,1.579658e+09,1.575740e+09,1.576885e+09,15000,900.00,16,900.00,1.576885e+09,False
...,...,...,...,...,...,...,...,...,...,...,...,...
4408,364109,FR,EUR,1.555348e+09,1.548589e+09,1.552673e+09,45000,62298.00,282,70527.02,1.555348e+09,True
4409,364399,US,USD,1.625934e+09,1.613865e+09,1.623342e+09,5000,956.00,55,956.00,1.623342e+09,False
4410,364426,US,USD,1.656083e+09,1.651769e+09,1.654614e+09,20000,159167.00,1960,159167.00,1.654614e+09,False
4411,364525,US,USD,1.618021e+09,1.612837e+09,1.612841e+09,5000,1.00,1,1.00,1.612841e+09,False


In [127]:
final_kickstarter_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4413 entries, 0 to 4412
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   bgg_id            4413 non-null   int64         
 1   country           4413 non-null   object        
 2   currency          4413 non-null   object        
 3   deadline          4413 non-null   datetime64[ns]
 4   created_at        4413 non-null   datetime64[ns]
 5   launched_at       4413 non-null   datetime64[ns]
 6   goal              4413 non-null   int64         
 7   pledged           4413 non-null   float64       
 8   backers_count     4413 non-null   int64         
 9   usd_pledged       4413 non-null   float64       
 10  state_changed_at  4413 non-null   datetime64[ns]
 11  successful        4413 non-null   bool          
dtypes: bool(1), datetime64[ns](4), float64(2), int64(3), object(2)
memory usage: 383.7+ KB


In [123]:
final_kickstarter_table = cjf.unix_to_datetime(df_clean_bgg_id_ks_slug,['deadline','created_at','launched_at','state_changed_at'])

In [124]:
final_kickstarter_table

,bgg_id,country,currency,deadline,created_at,launched_at,goal,pledged,backers_count,usd_pledged,state_changed_at,successful
0,37,NL,EUR,2019-03-01 21:00:00,2019-01-27 16:02:07,2019-02-14 09:32:29,4750,14132.00,161,16021.28,2019-03-01 21:00:00,True
1,46,US,USD,2017-11-27 04:00:00,2017-08-30 03:07:17,2017-10-19 18:14:04,18446,76254.00,980,76254.00,2017-11-27 04:00:00,True
2,196,US,USD,2013-07-10 13:37:56,2013-04-16 15:19:22,2013-06-10 13:37:56,9000,53724.58,1447,53724.58,2013-07-10 13:37:56,True
3,257,US,USD,2015-11-14 04:00:00,2015-10-09 15:52:21,2015-10-13 19:01:01,40000,210086.50,2711,210086.50,2015-11-14 04:00:14,True
4,470,US,USD,2020-01-22 02:00:00,2019-12-07 17:32:49,2019-12-20 23:43:12,15000,900.00,16,900.00,2019-12-20 23:43:13,False
...,...,...,...,...,...,...,...,...,...,...,...,...
4408,364109,FR,EUR,2019-04-15 17:00:00,2019-01-27 11:38:31,2019-03-15 18:00:03,45000,62298.00,282,70527.02,2019-04-15 17:00:00,True
4409,364399,US,USD,2021-07-10 16:25:19,2021-02-20 23:45:36,2021-06-10 16:25:19,5000,956.00,55,956.00,2021-06-10 16:25:20,False
4410,364426,US,USD,2022-06-24 14:57:00,2022-05-05 16:42:39,2022-06-07 14:57:00,20000,159167.00,1960,159167.00,2022-06-07 14:57:05,False
4411,364525,US,USD,2021-04-10 02:15:09,2021-02-09 02:20:02,2021-02-09 03:15:09,5000,1.00,1,1.00,2021-02-09 03:15:09,False


In [126]:
sf.build_table(engine=engine, table_name='clean_ks-with_bbg_id',dataframe=final_kickstarter_table,schema=schema)

The clean_ks-with_bbg_id table was imported successfully.


In [119]:
#df_clean_bgg_id_ks_slug.groupby('bgg_id').count()['created_at'].sort_values(ascending=False)